In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import os

In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
imdbdata = pd.read_csv('c:/Users/Ruchi/IMDB Dataset.csv')


In [7]:
from sklearn import preprocessing
sentimentlabels = preprocessing.LabelEncoder()
#to convert positive to 1 and negative to 0
imdbdata['sentiment'] = sentimentlabels.fit_transform(imdbdata['sentiment'])


In [8]:
#To check if changes have been made
imdbdata.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
import warnings
warnings.simplefilter('ignore')

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(list(imdbdata['review']))

all_ngrams = vectorizer.get_feature_names()
num_ngrams = min(50, len(all_ngrams))
all_counts = x.sum(axis=0).tolist()[0]

all_ngrams, all_counts = zip(*[(n, c) for c, n in sorted(zip(all_counts, all_ngrams), reverse=True)])
ngrams = all_ngrams[:num_ngrams]
counts = all_counts[:num_ngrams]

idx = np.arange(num_ngrams)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

def vectorise(texts, labels):
    kwargs = {
        'ngram_range' : (1, 2),
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : 'word',
        'min_df' : 2,
    }
    
    t_v = TfidfVectorizer(**kwargs)
    changed_text = t_v.fit_transform(texts)
    
       
    selector = SelectKBest(f_classif, k=min(20000, changed_text.shape[1]))
    selector.fit(changed_text, labels)
    changed_text = selector.transform(changed_text).astype('float32')
    return changed_text

v_data = vectorise(imdbdata['review'], imdbdata['sentiment'])

In [12]:
v_data.shape

(50000, 20000)

In [13]:
tr_texts = TfidfVectorizer().fit_transform(imdbdata['review'])
tr_texts.shape

(50000, 101895)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X = v_data.toarray()
y = (np.array(imdbdata['sentiment']))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
print("Train dataset shape: {0}, \nTest dataset shape: {1}".format(X_train.shape, X_test.shape))

Train dataset shape: (40000, 20000), 
Test dataset shape: (10000, 20000)


In [17]:
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout

In [19]:
model = keras.Sequential()

model.add(Dropout(0.2, X_train.shape[1:]))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
    
model.add(Dense(1, activation='sigmoid'))


In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]

In [22]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

Epoch 1/2
625/625 [==============================] - 19s 30ms/step - loss: 0.3089 - accuracy: 0.8901 - val_loss: 0.1938 - val_accuracy: 0.9268
Epoch 2/2
625/625 [==============================] - 17s 27ms/step - loss: 0.1621 - accuracy: 0.9377 - val_loss: 0.1819 - val_accuracy: 0.9311


In [23]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

313/313 [==============================] - 2s 6ms/step - loss: 0.1819 - accuracy: 0.9311
Test loss: 0.18185819685459137
Test accuracy: 0.9311000108718872


In [27]:
model.save('model_cnn.h5', include_optimizer=True)
json_string = model.to_json()
f = open("model_cnn_architecture.json",'w')
f.write(json_string)
f.close()

